In [1]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

C:\Users\dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import pandas as pd
import numpy as np

In [6]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [7]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [8]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [9]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'C:/Users/dell/Desktop/ai class/case studies/HAR/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [10]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'C:/Users/dell/Desktop/ai class/case studies/HAR/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [11]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [12]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [13]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [14]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [15]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [16]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [20]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [17]:
# https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

def model_creation(neurons,dropout_rate,epochs):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(timesteps, input_dim)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_classes, activation='sigmoid'))
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [21]:
model = KerasClassifier(build_fn=model_creation, epochs=20, batch_size=10, verbose=2)
neurons = [32,64,96]
dropout=[0.2,0.4,0.6,0.8]
params= dict(dropout_rate=dropout,neurons=neurons)
grid = GridSearchCV(estimator=model,param_grid=params,cv=3,verbose=2)
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] dropout_rate=0.2, neurons=32 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
 - 71s - loss: 1.2812 - acc: 0.4615
Epoch 2/20
 - 71s - loss: 1.0229 - acc: 0.5491
Epoch 3/20
 - 68s - loss: 0.8031 - acc: 0.6503
Epoch 4/20
 - 68s - loss: 0.6979 - acc: 0.6964
Epoch 5/20
 - 65s - loss: 0.5977 - acc: 0.7635
Epoch 6/20
 - 61s - loss: 0.5276 - acc: 0.8088
Epoch 7/20
 - 64s - loss: 0.4454 - acc: 0.8484
Epoch 8/20
 - 60s - loss: 0.3916 - acc: 0.8676
Epoch 9/20
 - 60s - loss: 0.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 21.3min remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
 - 65s - loss: 1.2807 - acc: 0.4509
Epoch 2/20
 - 64s - loss: 1.0298 - acc: 0.5595
Epoch 3/20
 - 60s - loss: 0.8684 - acc: 0.6601
Epoch 4/20
 - 61s - loss: 0.7402 - acc: 0.7372
Epoch 5/20
 - 86s - loss: 0.6402 - acc: 0.7980
Epoch 6/20
 - 115s - loss: 0.4616 - acc: 0.8708
Epoch 7/20
 - 83s - loss: 0.3667 - acc: 0.8957
Epoch 8/20
 - 61s - loss: 0.2998 - acc: 0.9112
Epoch 9/20
 - 63s - loss: 0

...

Epoch 5/20
 - 104s - loss: 0.7730 - acc: 0.6446
Epoch 6/20
 - 108s - loss: 0.6839 - acc: 0.6858
Epoch 7/20
 - 60s - loss: 0.6093 - acc: 0.7552
Epoch 8/20
 - 60s - loss: 0.4708 - acc: 0.8319
Epoch 9/20
 - 60s - loss: 0.3752 - acc: 0.8794
Epoch 10/20
 - 61s - loss: 0.3324 - acc: 0.8876
Epoch 11/20
 - 62s - loss: 0.2967 - acc: 0.9004
Epoch 12/20
 - 60s - loss: 0.2410 - acc: 0.9215
Epoch 13/20
 - 60s - loss: 0.2253 - acc: 0.9300
Epoch 14/20
 - 61s - loss: 0.2314 - acc: 0.9223
Epoch 15/20
 - 60s - loss: 0.1951 - acc: 0.9327
Epoch 16/20
 - 59s - loss: 0.1787 - acc: 0.9343
Epoch 17/20
 - 60s - loss: 0.1956 - acc: 0.9319
Epoch 18/20
 - 61s - loss: 0.1926 - acc: 0.9325
Epoch 19/20
 - 61s - loss: 0.1805 - acc: 0.9374
Epoch 20/20
 - 60s - loss: 0.1623 - acc: 0.9382
[CV] ..................... dropout_rate=0.6, neurons=64, total=22.5min
[CV] dropout_rate=0.6, neurons=96 ....................................
_________________________________________________________________
Layer (type)               

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_29 (LSTM)               (None, 32)                5376      
_________________________________________________________________
dropout_29 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
 - 87s - loss: 1.5411 - acc: 0.3361
Epoch 2/20
 - 86s - loss: 1.2791 - acc: 0.4579
Epoch 3/20
 - 86s - loss: 1.1865 - acc: 0.5003
Epoch 4/20
 - 82s - loss: 1.0912 - acc: 0.5452
Epoch 5/20
 - 77s - loss: 0.9731 - acc: 0.5938
Epoch 6/20
 - 78s - loss: 0.9565 - acc: 0.5931
Epoch 7/20
 - 83s - loss: 0.9206 - acc: 0.5929
Epoch 8/20
 - 80s - loss: 0.8842 - acc: 0.6117
Epoch 9/20
 - 79s - loss: 0.

Epoch 5/20
 - 69s - loss: 0.9691 - acc: 0.5826
Epoch 6/20
 - 77s - loss: 0.9313 - acc: 0.5759
Epoch 7/20
 - 98s - loss: 1.0585 - acc: 0.5502
Epoch 8/20
 - 79s - loss: 0.8844 - acc: 0.6022
Epoch 9/20
 - 81s - loss: 0.8107 - acc: 0.6308
Epoch 10/20
 - 84s - loss: 0.7814 - acc: 0.6579
Epoch 11/20
 - 86s - loss: 0.7007 - acc: 0.6950
Epoch 12/20
 - 84s - loss: 0.6524 - acc: 0.7452
Epoch 13/20
 - 83s - loss: 0.5471 - acc: 0.7936
Epoch 14/20
 - 95s - loss: 0.4794 - acc: 0.8376
Epoch 15/20
 - 106s - loss: 0.4811 - acc: 0.8529
Epoch 16/20
 - 105s - loss: 0.3647 - acc: 0.8898
Epoch 17/20
 - 99s - loss: 0.3220 - acc: 0.8982
Epoch 18/20
 - 101s - loss: 0.3652 - acc: 0.8931
Epoch 19/20
 - 112s - loss: 0.3598 - acc: 0.9004
Epoch 20/20
 - 95s - loss: 0.3258 - acc: 0.9004
[CV] ..................... dropout_rate=0.8, neurons=64, total=29.4min
[CV] dropout_rate=0.8, neurons=96 ....................................
_________________________________________________________________
Layer (type)             

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 887.5min finished


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 96)                40704     
_________________________________________________________________
dropout_37 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 6)                 582       
Total params: 41,286
Trainable params: 41,286
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
 - 143s - loss: 1.1991 - acc: 0.4746
Epoch 2/20
 - 130s - loss: 0.7650 - acc: 0.6636
Epoch 3/20
 - 132s - loss: 0.5724 - acc: 0.7643
Epoch 4/20
 - 134s - loss: 0.4186 - acc: 0.8502
Epoch 5/20
 - 134s - loss: 0.2913 - acc: 0.9019
Epoch 6/20
 - 131s - loss: 0.2495 - acc: 0.9169
Epoch 7/20
 - 131s - loss: 0.1955 - acc: 0.9310
Epoch 8/20
 - 133s - loss: 0.1722 - acc: 0.9361
Epoch 9/20
 - 135s

In [23]:
print(grid_result.best_score_)

0.9226060931462393


In [24]:
print(grid_result.best_params_)

{'dropout_rate': 0.4, 'neurons': 96}
